In [2]:
# import packages
import pandas as pd
import math
import numpy as np
import statsmodels.api as sm
from scipy import stats
import statistics
import plotly.graph_objects as go
from scipy.stats import norm
from plotly.subplots import make_subplots
from sklearn.neighbors import KernelDensity

In [3]:
# Load the market prices
S_P100index = pd.read_excel("/files/exercises/Homeworks/HW4/TP4.xls", sheet_name = 'S&P100Index', skiprows = 4)
S_P100index.columns = ['date','S&P100','S&P500']
S_P100index = S_P100index.set_index('date')

# Load the stocks prices
S_P100const = pd.read_excel("/files/exercises/Homeworks/HW4/TP4.xls", sheet_name = 'S&P100Constituents', skiprows = 3)
S_P100const = S_P100const.drop([0])
S_P100const.rename(columns = {'Name': 'date'}, inplace=True)
S_P100const = S_P100const.set_index('date')
S_P100const = S_P100const.astype(float)

# Load the risk free rates
TBill3Months = pd.read_excel("/files/exercises/Homeworks/HW4/TP4.xls", sheet_name = 'TBill3Months', skiprows = 4)
TBill3Months.columns = ['date','US bill 3m']
TBill3Months = TBill3Months.set_index('date')

# Load something
FamaFrenchPortfolios = pd.read_excel("/files/exercises/Homeworks/HW4/TP4.xls", sheet_name = 'FamaFrenchPortfolios', skiprows = 21)
FamaFrenchPortfolios.columns = ['date','Small_Low BE/ME', 'Small_Med BE/ME', 'Small_High BE/ME','Big_Low BE/ME', 'Big_Med BE/ME', 'Big_High BE/ME']
FamaFrenchPortfolios = FamaFrenchPortfolios.set_index('date')

In [4]:
S_P100index.iloc[0:-1,0:].values

array([[192.13, 422.88],
       [192.31, 423.61],
       [195.36, 429.19],
       ...,
       [447.88, 882.5 ],
       [451.2 , 885.76],
       [460.55, 902.65]])

In [5]:
# Compute Arithmetic return for each market
AR_100 = pd.DataFrame((S_P100index.iloc[1:,0:].values - S_P100index.iloc[0:-1,0:].values) / S_P100index.iloc[0:-1,0:].values,  columns = ['S&P100','S&P500'])

# Compute Arithmetic return for each stocks
names_company = S_P100const.columns
AR_100_const = pd.DataFrame((S_P100const.iloc[1:,0:].values - S_P100const.iloc[0:-1,0:].values) / S_P100const.iloc[0:-1,0:].values, columns = names_company)

# Compute risk free rates (since it's given annualy we have to divide it by 100 and 52 to get weekly's rates)
AR_US3M = pd.DataFrame((TBill3Months.iloc[1:,0:].values / (100*52)),  columns = ['US 3 month'])

# Test of the CAPM : Time Series approach

## Exercice 1

In [6]:
# Compute the index of the return of interest
index_start = int(np.where(S_P100index.index == '1992-11-12')[0])
index_end = int(np.where(S_P100index.index == '2001-08-16')[0] - 1)

In [7]:
# Compute the beta of each stock thanks to the formula cov(ri,rm)/var(rm)
beta_stock = np.ones(AR_100_const.shape[1])
for i in range(0, AR_100_const.shape[1]):
    beta_stock[i] = np.cov(AR_100_const.iloc[index_start:index_end, i].values, AR_100.iloc[index_start:index_end, 1].values)[0, 1] / np.cov(AR_100_const.iloc[index_start:index_end, i].values, AR_100.iloc[index_start:index_end, 1].values)[1, 1]

pd.DataFrame(beta_stock, index = names_company, columns = ['Beta'])

,Beta
AES,1.012828
ALCOA,0.616923
ALLEGHENY TECHS.,0.702266
AMER.ELEC.PWR.,0.262885
AMER.EXPRESS,1.501120
...,...
WELLS FARGO & CO,1.124522
WEYERHAEUSER,0.713086
WILLIAMS COS.,0.817679
XEROX,1.349616


## Exercice 2

In [8]:
# Compute the alphas and betas of each stock thanks to the linear regression
nb_asset = len(AR_100_const.columns) # Compute the number of colums to deal with
betas = np.zeros(nb_asset) # Initialize an array with nb_asset entries to receive the beta
alphas = np.zeros(nb_asset) # Initialize an array with nb_asset entries to receive the alpha
std_alpha = np.zeros(nb_asset) # Initialize an array with nb_asset entries to receive the std of alpha

for i in range(0, nb_asset):
    zi =  AR_100_const.iloc[index_start:index_end, i].values - AR_US3M.iloc[index_start:index_end, 0].values
    zm = AR_100.iloc[index_start:index_end, 1].values - AR_US3M.iloc[index_start:index_end, 0].values
    X = sm.add_constant(zm)
    y = zi
    
    # Fit regression on the data of interest and save the alpha, beta and std of alpha
    reg = sm.OLS(endog = y, exog = X)
    results = reg.fit()
    betas[i] = results.params[1]
    alphas[i] = results.params[0]
    std_alpha[i] = results.bse[0]

In [9]:
# Compute the t-value of the alpha --> alpha/std_alpha (if absolut value < 1.96 --> can not reject H0 (alpha = 0))
pd.DataFrame(np.transpose(np.array([betas, alphas, std_alpha, np.around(abs(alphas/std_alpha), decimals=2)])), index = names_company, columns = ['beta', 'alpha', 'std_alpha', 't-value'])

,beta,alpha,std_alpha,t-value
AES,1.012350,0.004353,0.002591,1.68
ALCOA,0.616899,0.002267,0.002010,1.13
ALLEGHENY TECHS.,0.703382,-0.001898,0.002084,0.91
AMER.ELEC.PWR.,0.262135,-0.000062,0.001372,0.05
AMER.EXPRESS,1.501551,0.001655,0.001557,1.06
...,...,...,...,...
WELLS FARGO & CO,1.124238,0.001355,0.001535,0.88
WEYERHAEUSER,0.713880,-0.000244,0.001858,0.13
WILLIAMS COS.,0.817565,0.002626,0.002032,1.29
XEROX,1.351780,-0.001567,0.002970,0.53


In [10]:
# Compute the number of stocks (over 95 = nb_asset) with alpha = 0 (statistically speaking)
np.sum(abs(alphas/std_alpha) < 1.96)

90

In [11]:
# Fit regression on the remaining data and save the r-squared of each stocks
index_start2 = int(np.where(S_P100index.index == '2001-08-16')[0] - 1)
r2 = np.zeros(nb_asset)

for i in range(0, nb_asset):
    zi =  AR_100_const.iloc[index_start2:, i].values - AR_US3M.iloc[index_start2:, 0].values
    zm = AR_100.iloc[index_start2:, 1].values - AR_US3M.iloc[index_start2:, 0].values
    X = sm.add_constant(zm)
    y = zi
    
    reg = sm.OLS(endog = y, exog = X, missing='drop')
    results = reg.fit()
    r2[i] = results.rsquared

In [12]:
# Compute the r-squared for each stocks
pd.DataFrame(np.around(r2, decimals=2), index = names_company, columns = ['R-squared']) 

,R-squared
AES,0.19
ALCOA,0.64
ALLEGHENY TECHS.,0.39
AMER.ELEC.PWR.,0.05
AMER.EXPRESS,0.73
...,...
WELLS FARGO & CO,0.39
WEYERHAEUSER,0.53
WILLIAMS COS.,0.15
XEROX,0.29


In [13]:
# Compute the average r-squared of each stocks
statistics.mean(np.around(r2, decimals=2))

0.32789473684210524

# Test of the CAPM : Cross-Section Approach (Fama MacBeth Procedure)

### Exercise 1

In [14]:
# Compute the index of the return of interest
index_start = int(np.where(S_P100index.index == '1992-11-12')[0])
index_end = int(np.where(S_P100index.index == '1996-11-14')[0] - 1)

# Compute the beta of each stock thanks to the formula cov(ri,rm)/var(rm)
beta_stock = np.ones(AR_100_const.shape[1])
for i in range(0, AR_100_const.shape[1]):
    beta_stock[i] = np.cov(AR_100_const.iloc[index_start:index_end, i].values, AR_100.iloc[index_start:index_end, 1].values)[0, 1] / np.cov(AR_100_const.iloc[index_start:index_end, i].values, AR_100.iloc[index_start:index_end, 1].values)[1, 1]

pd.DataFrame(beta_stock, index = names_company, columns = ['Beta'])

,Beta
AES,0.771203
ALCOA,0.801033
ALLEGHENY TECHS.,0.690779
AMER.ELEC.PWR.,0.565845
AMER.EXPRESS,1.432065
...,...
WELLS FARGO & CO,1.127970
WEYERHAEUSER,0.867814
WILLIAMS COS.,0.740716
XEROX,0.815155


### Exercise 2

In [15]:
# Sort and split betas into 10 sets
beta_alphabet = pd.DataFrame(beta_stock, index = names_company, columns = ['Beta'])
beta_sorted = beta_alphabet.sort_values(by=['Beta'])
Portfolio_1 = beta_sorted[0:10].index.tolist()
Portfolio_2 = beta_sorted[10:20].index.tolist()
Portfolio_3 = beta_sorted[20:30].index.tolist()
Portfolio_4 = beta_sorted[30:40].index.tolist()
Portfolio_5 = beta_sorted[40:50].index.tolist()
Portfolio_6 = beta_sorted[50:60].index.tolist()
Portfolio_7 = beta_sorted[60:70].index.tolist()
Portfolio_8 = beta_sorted[70:80].index.tolist()
Portfolio_9 = beta_sorted[80:90].index.tolist()
Portfolio_10 = beta_sorted[90:].index.tolist()

### Exercise 3

In [17]:
# Compute the index of the return of interest
index_start = int(np.where(S_P100index.index == '1996-11-21')[0])
index_end = int(np.where(S_P100index.index == '2000-11-16')[0] - 1)

# Compute the beta of each stock thanks to the formula cov(ri,rm)/var(rm)
beta_stock2 = np.ones(AR_100_const.shape[1])
for i in range(0, AR_100_const.shape[1]):
    beta_stock2[i] = np.cov(AR_100_const.iloc[index_start:index_end, i].values, AR_100.iloc[index_start:index_end, 1].values)[0, 1] / np.cov(AR_100_const.iloc[index_start:index_end, i].values, AR_100.iloc[index_start:index_end, 1].values)[1, 1]

beta_stock2 = pd.DataFrame(beta_stock2, index = names_company, columns = ['Beta'])

In [18]:
beta_PF1 = statistics.mean(beta_stock2.loc[Portfolio_1]['Beta'])
beta_PF2 = statistics.mean(beta_stock2.loc[Portfolio_2]['Beta'])
beta_PF3 = statistics.mean(beta_stock2.loc[Portfolio_3]['Beta'])
beta_PF4 = statistics.mean(beta_stock2.loc[Portfolio_4]['Beta'])
beta_PF5 = statistics.mean(beta_stock2.loc[Portfolio_5]['Beta'])
beta_PF6 = statistics.mean(beta_stock2.loc[Portfolio_6]['Beta'])
beta_PF7 = statistics.mean(beta_stock2.loc[Portfolio_7]['Beta'])
beta_PF8 = statistics.mean(beta_stock2.loc[Portfolio_8]['Beta'])
beta_PF9 = statistics.mean(beta_stock2.loc[Portfolio_9]['Beta'])
beta_PF10 = statistics.mean(beta_stock2.loc[Portfolio_10]['Beta'])
pd.DataFrame([beta_PF1,beta_PF2,beta_PF3,beta_PF4,beta_PF5,beta_PF6,beta_PF7,beta_PF8,beta_PF9,beta_PF10], index=[list(range(1,11))], columns=['Average beta per set'])

,Average beta per set
1,0.500247
2,0.761830
3,0.704776
4,0.722732
5,0.889147
6,0.691271
7,0.842323
8,1.196763
9,1.391838
10,1.643725


### Exercise 4

# Alternative to CAPM : The Fama-French (1992) Model

In [19]:
#FamaFrenchPortfolios = FamaFrenchPortfolios/100

### Exercise 1

In [20]:
# Computation of SMB and HML
SMB = np.mean(FamaFrenchPortfolios.iloc[0:,0:3].values, 1)-np.mean(FamaFrenchPortfolios.iloc[0:,3:].values, 1)
HML = np.mean(FamaFrenchPortfolios.iloc[0:, [2,5]].values, 1) - np.mean(FamaFrenchPortfolios.iloc[0:, [0,3]].values, 1)

### Exercise 2

In [21]:
# Compute the index of the return of interest
index_start = int(np.where(FamaFrenchPortfolios.index == '1992-11-12')[0])
index_end = int(np.where(FamaFrenchPortfolios.index == '2001-08-16')[0] - 1)

# Compute the alphas and betas of each portfolio thanks to the linear regression
nb_PF = len(FamaFrenchPortfolios.columns) # Compute the number of colums to deal with
betas = np.zeros(nb_PF) # Initialize an array with nb_asset entries to receive the beta
alphas = np.zeros(nb_PF) # Initialize an array with nb_asset entries to receive the alpha
s_SMB = np.zeros(nb_PF) # Initialize an array with nb_asset entries to receive the s
h__HML = np.zeros(nb_PF) # Initialize an array with nb_asset entries to receive the h
std_alpha = np.zeros(nb_PF) # Initialize an array with nb_asset entries to receive the std of alpha

for i in range(0, nb_PF):
    zi =  FamaFrenchPortfolios.iloc[index_start:index_end, i].values - AR_US3M.iloc[index_start:index_end, 0].values
    zm = AR_100.iloc[index_start:index_end, 1].values - AR_US3M.iloc[index_start:index_end, 0].values
    X = np.transpose([zm, SMB[index_start:index_end], HML[index_start:index_end]])
    X = sm.add_constant(X)
    y = zi
    
    # Fit regression on the data of interest and save the alpha, beta and std of alpha
    reg = sm.OLS(endog = y, exog = X)
    results = reg.fit() 
    betas[i] = results.params[1]
    alphas[i] = results.params[0]
    s_SMB[i] = results.params[2]
    h__HML[i] = results.params[3]
    std_alpha[i] = results.bse[0]

In [22]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.293
Model:                            OLS   Adj. R-squared:                  0.288
Method:                 Least Squares   F-statistic:                     62.36
Date:                Sat, 17 Oct 2020   Prob (F-statistic):           9.36e-34
Time:                        11:52:42   Log-Likelihood:                -787.65
No. Observations:                 456   AIC:                             1583.
Df Residuals:                     452   BIC:                             1600.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.4173      0.065      6.432      0.000       0.290       0.545
x1             0.4713      3.139      0.150      0.881      -5.697       6.640
x2            -0.5626      0.044    -12.781      0.000      -0.649      -0.476
x3            -0.2992      0.039     -7.649      0.000      -0.376      -0.222
==============================================================================
Omnibus:                       32.553   Durbin-Watson:                   1.667
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               75.165
Skew:                          -0.369   Prob(JB):                     4.77e-17
Kurtosis:                       4.847   Cond. No.                         91.4
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [23]:
# Compute the index of the return of interest
index_start = int(np.where(FamaFrenchPortfolios.index == '2001-08-16')[0])
index_end = int(len(FamaFrenchPortfolios)-1)

# Compute the alphas and betas of each portfolio thanks to the linear regression
nb_PF = len(FamaFrenchPortfolios.columns) # Compute the number of colums to deal with
betas = np.zeros(nb_PF) # Initialize an array with nb_asset entries to receive the beta
alphas = np.zeros(nb_PF) # Initialize an array with nb_asset entries to receive the alpha
s_SMB = np.zeros(nb_PF) # Initialize an array with nb_asset entries to receive the s
h__HML = np.zeros(nb_PF) # Initialize an array with nb_asset entries to receive the h
std_alpha = np.zeros(nb_PF) # Initialize an array with nb_asset entries to receive the std of alpha

for i in range(0, nb_PF):
    zi =  FamaFrenchPortfolios.iloc[index_start:index_end, i].values - AR_US3M.iloc[index_start:index_end, 0].values
    zm = AR_100.iloc[index_start:index_end, 1].values - AR_US3M.iloc[index_start:index_end, 0].values
    X = np.transpose([zm, SMB[index_start:index_end], HML[index_start:index_end]])
    X = sm.add_constant(X)
    y = zi
    
    # Fit regression on the data of interest and save the alpha, beta and std of alpha
    reg = sm.OLS(endog = y, exog = X)
    results = reg.fit() 
    betas[i] = results.params[1]
    alphas[i] = results.params[0]
    s_SMB[i] = results.params[2]
    h__HML[i] = results.params[3]
    std_alpha[i] = results.bse[0]

In [24]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.264
Model:                            OLS   Adj. R-squared:                  0.228
Method:                 Least Squares   F-statistic:                     7.295
Date:                Sat, 17 Oct 2020   Prob (F-statistic):           0.000294
Time:                        11:52:43   Log-Likelihood:                -161.15
No. Observations:                  65   AIC:                             330.3
Df Residuals:                      61   BIC:                             339.0
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0678      0.377     -0.180      0.858      -0.822       0.686
x1            -6.7190     12.675     -0.530      0.598     -32.064      18.626
x2            -0.7647      0.277     -2.764      0.008      -1.318      -0.211
x3            -0.7017      0.266     -2.636      0.011      -1.234      -0.169
==============================================================================
Omnibus:                       35.995   Durbin-Watson:                   1.531
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              117.031
Skew:                          -1.588   Prob(JB):                     3.86e-26
Kurtosis:                       8.755   Cond. No.                         60.5
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Exercise 3

In [25]:
# Compute the index of the return of interest
index_start = int(np.where(FamaFrenchPortfolios.index == '1992-11-12')[0])
index_end = int(np.where(FamaFrenchPortfolios.index == '2001-08-16')[0] - 1)

# Compute the alphas and betas of each portfolio thanks to the linear regression
nb_PF = len(FamaFrenchPortfolios.columns) # Compute the number of colums to deal with
betas = np.zeros(nb_PF) # Initialize an array with nb_asset entries to receive the beta
alphas = np.zeros(nb_PF) # Initialize an array with nb_asset entries to receive the alpha
std_alpha = np.zeros(nb_PF) # Initialize an array with nb_asset entries to receive the std of alpha

for i in range(0, nb_PF):
    zi =  FamaFrenchPortfolios.iloc[index_start:index_end, i].values - AR_US3M.iloc[index_start:index_end, 0].values
    zm = AR_100.iloc[index_start:index_end, 1].values - AR_US3M.iloc[index_start:index_end, 0].values
    X = sm.add_constant(zm)
    y = zi
    
    # Fit regression on the data of interest and save the alpha, beta and std of alpha
    reg = sm.OLS(endog = y, exog = X)
    results = reg.fit() 
    betas[i] = results.params[1]
    alphas[i] = results.params[0]
    std_alpha[i] = results.bse[0]

In [26]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                  0.009
Method:                 Least Squares   F-statistic:                     5.352
Date:                Sat, 17 Oct 2020   Prob (F-statistic):             0.0211
Time:                        11:52:43   Log-Likelihood:                -863.95
No. Observations:                 456   AIC:                             1732.
Df Residuals:                     454   BIC:                             1740.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3055      0.076      4.034      0.000       0.157       0.454
x1             8.4082      3.635      2.313      0.021       1.265      15.551
==============================================================================
Omnibus:                       12.693   Durbin-Watson:                   2.084
Prob(Omnibus):                  0.002   Jarque-Bera (JB):               17.706
Skew:                          -0.242   Prob(JB):                     0.000143
Kurtosis:                       3.835   Cond. No.                         48.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""